In [8]:
from PPO_mountaincar import *
import gym
import pickle


In [2]:
env = gym.make("MountainCar-v0")
agent1 = PPO(n_actions=env.action_space.n, input_dims=env.observation_space.shape)
agent1.load_models("model/best_actor_model_MC")
agent2 = PPO(n_actions=env.action_space.n, input_dims=env.observation_space.shape)
agent2.load_models("model/half_actor_model_MC")


... loading models ...
... loading models ...


In [3]:
K = 30
 
trajectory_record = [] # [(t_1,t_2)_1,...,(t_1,t_2)_K]
reward_record = []  # [(R_1,R_2)_1,...,(R_1,R_2)_K]

def softmax(x,y):
    return np.exp(x)/(np.exp(x)+np.exp(y))

for k in range(K):
    # agent1
    state, _ = env.reset()
    done = False
    cum_reward1 = 0
    cum_reward2 = 0
    trajectory_buffer1 = []  
    trajectory_buffer2 = [] 
    while not done:
        action, probs, value = agent1.choose_action(state)
        trajectory_buffer1.append([[state[0],state[1]], action]) 
        state_, reward, terminated, truncated, _ = env.step(action)
        state = state_
        cum_reward1 += reward
        done = terminated

    # agent2
    state, _ = env.reset()
    done = False

    while not done:
        action, probs, value = agent2.choose_action(state)
        trajectory_buffer2.append([[state[0],state[1]], action]) 
        state_, reward, terminated, truncated, _ = env.step(action)
        state = state_
        cum_reward2 += reward
        done = terminated

    trajectory_record.append([trajectory_buffer1,trajectory_buffer2])
    reward_record.append([cum_reward1, cum_reward2])

probs = [[softmax(x[0],x[1]),softmax(x[1],x[0])] for x in reward_record]
label = []
for prob in probs:
    label.append(0 if prob[0]>prob[1] else 1)


/Users/pikkun0907/Library/CloudStorage/OneDrive-epfl.ch/2nd semester/RL/RL_project_epfl/RL_PPO/PPO_mountaincar.py:129: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  state = torch.tensor([observation], dtype=torch.float).to(self.actor.device)
/opt/anaconda3/envs/epfl/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


## save traj

In [9]:
trajectory_record # K x n_policy(good one or bad one) x stepsize
label # 0 -> good one, 1 -> bad one

with open("trajectory_record.pkl", "wb") as f:
    pickle.dump({"trajectory": trajectory_record, "label": label}, f)

## load traj

In [10]:
with open("trajectory_record.pkl", "rb") as f:
    data = pickle.load(f)

trajectory_record_from_loading = data["trajectory"]
label_from_loading = data["label"]